In [1]:
# Python script to train a neural network using Keras library.

import os
import zipfile

os.environ['KMP_DUPLICATE_LIB_OK']='True'

import pandas
import numpy as np
import random
import sys

# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.layers import LSTM
# from tensorflow.keras.layers import Dropout
# from tensorflow.keras.layers import Conv1D
# from tensorflow.keras.layers import MaxPooling1D

# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.models import load_model
# from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

# random.seed(datetime.now())
random.seed(0)

In [2]:
# model_type
# 1: both left and right sides of time series are padded
# 2: only left side of time series is padded

# Get command line parameters
# model_type = int(sys.argv[1])
# kk = int(sys.argv[2]) # index for NN
model_type = 1
kk = 1

print('Train a classifier of type {} with index {}'.format(model_type, kk))



# Set size of training library and length of time series
(lib_size, ts_len) = (200000, 1500) # Or (500000, 500) for the 500-classifier


# keeps track of training metrics
f1_name = 'training_results_{}_{}.txt'.format(kk, model_type)
f2_name = 'training_results_{}_{}.csv'.format(kk, model_type)

f_results= open(f1_name, "w")
f_results2 = open(f2_name, "w")


if model_type==1:
    pad_left = 225 if ts_len==500 else 725
    pad_right = 225 if ts_len==500 else 725

if model_type==2:
    pad_left = 450 if ts_len==500 else 1450
    pad_right = 0
    

# get zipfile of time series 
print('Load in zip file containing training data')
zf = zipfile.ZipFile('../training_data/output_full/ts_{}/combined/output_resids.zip'.format(ts_len))
text_files = zf.infolist()
sequences = list()


print('Extract time series from zip file')
tsid_vals = np.arange(1,lib_size+1)
for tsid in tsid_vals:
    df = pandas.read_csv(zf.open('output_resids/resids'+str(tsid)+'.csv'))
    values = df[['Residuals']].values
    sequences.append(values)

sequences = np.array(sequences)

# Get target labels for each data sample
df_targets = pandas.read_csv('../training_data/output_full/ts_{}/combined/labels.csv'.format(ts_len),
                          index_col='sequence_ID')

# train/validation/test split denotations
df_groups = pandas.read_csv('../training_data/output_full/ts_{}/combined/groups.csv'.format(ts_len),
                            index_col='sequence_ID')


Train a classifier of type 1 with index 1
Load in zip file containing training data
Extract time series from zip file


In [3]:
final_seq = sequences

# apply train/test/validation labels
train = [final_seq[i] for i, tsid in enumerate(tsid_vals) if df_groups['dataset_ID'].loc[tsid]==1]
validation = [final_seq[i] for i, tsid in enumerate(tsid_vals) if df_groups['dataset_ID'].loc[tsid]==2]
test = [final_seq[i] for i, tsid in enumerate(tsid_vals) if df_groups['dataset_ID'].loc[tsid]==3]


train_target = [df_targets['class_label'].loc[tsid] for i, tsid in enumerate(tsid_vals) if df_groups['dataset_ID'].loc[tsid]==1]
validation_target = [df_targets['class_label'].loc[tsid] for i, tsid in enumerate(tsid_vals) if df_groups['dataset_ID'].loc[tsid]==2]
test_target = [df_targets['class_label'].loc[tsid] for i, tsid in enumerate(tsid_vals) if df_groups['dataset_ID'].loc[tsid]==3]  



In [8]:
import pickle
data_dic = {
    "train": train,
    "train_target": train_target,
    "validation": validation,
    "validation_target": validation_target,
    "test": test,
    "test_target": test_target
}

with open("preproccessed_data_uncensored.pickle", "wb") as f:
    pickle.dump(data_dic, f)



In [1]:
import pickle
with open("preproccessed_data_uncensored.pickle", "rb") as f:
    data_dic = pickle.load(f)


train = data_dic['train']
train_target = data_dic['train_target']
validation = data_dic['validation']
validation_target = data_dic['validation_target']
test = data_dic['test']
test_target = data_dic['test_target']

In [ ]:
import pickle
import pandas as pd
import numpy as np
import csv
print("loading data")
with open("preproccessed_data_uncensored.pickle", "rb") as f:
    data_dic = pickle.load(f)

train = data_dic['train']
train_target = data_dic['train_target']
validation = data_dic['validation']
validation_target = data_dic['validation_target']
test = data_dic['test']
test_target = data_dic['test_target']

In [14]:
def Xy_to_lookback(data, data_target, target_file, window_size=600):
    # X = []
    # y = []
    # wr = csv.writer(target_file, lineterminator = '\n')
    for i in range(len(data)):
        for j in range(len(data[i])-window_size-1):
            X = data[i][j:(j+window_size)]
            y = data_target[i]
            row = [[y]]
            row.extend(list(X))
            # wr.writerow(row)
            np.savetxt(target_file, row, delimiter=",")

f1_name = 'xy_train.csv'
f2_name = 'xy_validation.csv'
f3_name = 'xy_test.csv'

f1_results= open(f1_name, "w")
f2_results= open(f2_name, "w")
f3_results= open(f3_name, "w")





# print("processing train")
# Xy_to_lookback(train, train_target, f1_results)
# f1_results.flush()
# f1_results.close()
# print("processing validation")
# Xy_to_lookback(validation, validation_target, f2_results)
# f2_results.flush()
# f2_results.close()
print("processing test")
Xy_to_lookback(test, test_target, f3_results)


f3_results.flush()
f3_results.close()

processing test


KeyboardInterrupt: 

In [13]:
np.loadtxt("xy_test.csv")

ValueError: could not convert string '0,[0.02755083],[0.02436612],[0.0170411],[0.00749803],[0.00839069],[0.01426757],[-0.00560297],[-0.01 to float64 at row 0, column 1.

In [4]:
for x in test:
    if len(x) != 1500:
        print('error')

In [3]:
def Xy_to_lookback(data, data_target, window_size=600):
    X = []
    y = []
    for i in range(len(data)):
        for j in range(len(data[i])-window_size):
            X.append([data[j:(j+window_size)]])
            y.append(data_target[i])
    
    return np.array(X), np.array(y)

In [16]:
import csv
f1 = open("test.csv", "w")
wr = csv.writer(f1, lineterminator = '\n')
x = [1,2,3]
data = [222]
data.extend(x)
wr.writerow(data)
wr.writerow(data)
f1.flush()
f1.close()

In [11]:
import pandas as pd
d = pd.read_csv("test.csv")

In [12]:
d

,1,2,3
0,1,3,4


In [4]:
x_train, y_train = Xy_to_lookback(train, train_target)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/rezmiry/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
  File "/tmp/ipykernel_9711/2809543262.py", line 1, in <module>
  File "/tmp/ipykernel_9711/2864672604.py", line 6, in Xy_to_lookback
MemoryError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/rezmiry/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 2105, in showtraceback
  File "/home/rezmiry/venv/lib/python3.10/site-packages/IPython/core/ultratb.py", line 1396, in structured_traceback
  File "/home/rezmiry/venv/lib/python3.10/site-packages/IPython/core/ultratb.py", line 1287, in structured_traceback
  File "/home/rezmiry/venv/lib/python3.10/site-packages/IPython/core/ultratb.py", line 1140, in structured_traceback
  File "/home/rezmiry/venv/lib/python3.10/site-packages/IPython/core/ultratb.py", line 1030, in format_exception_as_a_whole
  File

In [2]:
import pickle
with open("./preproccessed_data.pickle", "rb") as f:
    data_dic = pickle.load(f)

In [3]:
train =data_dic['train']
validation =data_dic['validation']
test =data_dic['test']

train_target =data_dic['train_target']
validation_target =data_dic['validation_target']
test_target =data_dic['test_target']

In [7]:
window_size = 600
X = []
y = []
for i in range(len(train)):
    for j in range(len(train[i])-window_size):
        X.append([train[j:(j+window_size)]])
        y.append(train_target[i])
    break
        
